In [1]:
%%capture
!wget "https://repo1.maven.org/maven2/org/apache/spark/spark-sql-kafka-0-10_2.12/3.3.1/spark-sql-kafka-0-10_2.12-3.3.1.jar"
!wget "https://repo1.maven.org/maven2/org/apache/spark/spark-streaming-kafka-0-10_2.12/3.1.3/spark-streaming-kafka-0-10_2.12-3.1.3.jar"

In [2]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-10_2.12:3.1.3,org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.1 pyspark-shell'

In [14]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [15]:
spark = SparkSession.builder.appName("wordcount2").master("local").getOrCreate()
customSchema = StructType() \
 .add("timestamp", StringType(), True) \
 .add("value", StringType(), True)


In [16]:
lines = spark \
    .readStream \
    .option("header", True) \
    .schema(customSchema) \
    .csv("/home/jovyan/tmp/text")

In [17]:
lines.printSchema()

root
 |-- timestamp: string (nullable = true)
 |-- value: string (nullable = true)



In [18]:
'''lines = spark \
 	.readStream \
 	.format("csv") \
 	.option("header",True) \
    .schema(customSchema) \
 	.load("/home/jovyan/")'''


'lines = spark  \t.readStream  \t.format("csv")  \t.option("header",True)     .schema(customSchema)  \t.load("/home/jovyan/")'

In [19]:
#lines.printSchema()

In [20]:
 words = lines.select(
        explode(split(lines.value, ' ')).alias('word'),
        lines.timestamp
    )

In [21]:
words.printSchema()

root
 |-- word: string (nullable = false)
 |-- timestamp: string (nullable = true)



In [22]:
 # Group the data by window and word and compute the count of each group
windowedCounts = words.groupBy(
    window(words.timestamp, "10 minutes","5 minutes"),
    words.word
).count().orderBy('window')

In [23]:
windowedCounts.printSchema()

root
 |-- window: struct (nullable = true)
 |    |-- start: timestamp (nullable = true)
 |    |-- end: timestamp (nullable = true)
 |-- word: string (nullable = false)
 |-- count: long (nullable = false)



In [ ]:
# Start running the query that prints the windowed word counts to the console
query = windowedCounts\
    .writeStream\
    .outputMode('complete')\
    .format('console')\
    .option('truncate', 'false')\
    .start()
query.awaitTermination()